# Demonstration View

View a demonstration by sliding through the frames.
This also plots the z height.

In [ ]:
%matplotlib notebook
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt

folder_format = "lukas"

#recording = "bolt_recordings/episode_3"
#recording = "stack_recordings/episode_118"
recording = "/media/kuka/Seagate Expansion Drive/kuka_recordings/flow/pick"
#recording = "block_yellow_recordings/episode_1"

if folder_format == 'max':
    state_recording_fn = "./{}/episode_1.npz".format(recording)
    state_recording = np.load(state_recording_fn)
    ee_positions = state_recording["ee_positions"]
    flow_recording_fn = "./{}/episode_1_img.npz".format(recording)
    flow_recording = np.load(flow_recording_fn)["img"]
else:
    state_recording_fn = "{}/episode_0.npz".format(recording)
    state_recording = np.load(state_recording_fn)["robot_state_full"]
    ee_positions = state_recording[:,:3]
    print(state_recording.shape)
    flow_recording_fn = "{}/episode_0.npz".format(recording)
    flow_recording = np.load(flow_recording_fn)["rgb_unscaled"]
num_frames = flow_recording.shape[0]-1


x = np.linspace(0, 2 * np.pi)
fig, (ax,ax2,ax3) = plt.subplots(3,1,figsize=(8, 6))
line = ax.imshow(image_recording[0])
ax.set_axis_off()

#ax2.plot(ee_positions[:,0],label="x")
#ax2.plot(ee_positions[:,1],label="y")
ax2.plot(ee_positions[:,2],label="z")
ax2.set_ylabel("z position")
ax2.set_xlabel("frame number")
ax2_2 = ax2.twinx()
ax2_2.plot(state_recording["gripper_states"])
vline2 = ax2.axvline(x=2,color="k")

ee_angles = state_recording["ee_angles"]
agl_diff = np.diff(ee_angles, axis=0)
pos_diff = np.diff(ee_positions, axis=0)
ax3.plot(pos_diff[:,0],label="x")
ax3.plot(pos_diff[:,1],label="y")
ax3.plot(pos_diff[:,2],label="z")
ax3.plot(agl_diff[:]/10,label="a")
ax3.plot(np.linalg.norm(pos_diff,axis=1),label="|m|")
vline3 = ax3.axvline(x=2,color="k")
ax3.legend()


ax2.legend()

def update(w):
    vline2.set_data([w,w],[0,1])
    vline3.set_data([w,w],[0,1])
    line.set_data(image_recording[w])
    fig.canvas.draw_idle()
    
slider_w = widgets.IntSlider(min=0,max=num_frames,step=1,value=0,
                             layout=Layout(width='70%'))
interact(update,w=slider_w)

In [ ]:
list(state_recording.keys())
gripper_states = state_recording["gripper_states"]
print(gripper_states.astype(int))
diff = np.diff(gripper_states)
gripper_state_changes = np.where(diff)[0]+1

switch_frame = gripper_state_changes[0]

print(gripper_states[switch_frame-1], gripper_states[switch_frame])
print(switch_frame)

# Save images to Files

In [ ]:
import PIL
img_dir =  os.path.join(recording,'image')
os.makedirs(img_dir, exist_ok=True)

for i in range(image_recording.shape[0]):
    fn = os.path.join(img_dir, f'{i:03d}.png')
    PIL.Image.fromarray(image_recording[i]).save(fn)
print("done!")

# Mask View

Mask out the foreground object so that foreground specific flow can be calculated.

In [ ]:
fig, ax = plt.subplots(1,1)
line = ax.imshow(image_recording[25])
ax.set_axis_off()

def get_mask(frame, threshold, color_index=2):
    image = frame.copy()
    tmp = image[:,:,color_index] / np.linalg.norm(image,axis=2) 
    mask = tmp > threshold
    #mask = np.logical_not(mask) for yellow
    #print("threshold", threshold)
    return mask

def update(w,i):
    image = image_recording[i].copy()
    mask = get_mask(image, w/100)
    image[np.logical_not(mask)] = 255,255,255
    line.set_data(image)
    fig.canvas.draw_idle()
    
slider_w = widgets.IntSlider(min=0,max=100,step=1,value=60,
                             layout=Layout(width='70%'))
slider_i = widgets.IntSlider(min=0,max=num_frames,step=1,value=25,
                             layout=Layout(width='70%'))

interact(update,w=slider_w, i=slider_i)

In [ ]:
threshold = 0.6
masks = np.zeros(image_recording.shape[:3],dtype=bool)

if recording == "stack_recordings/episode_118":
    switch_frame = 65
elif recording == "stack_recordings/episode_129":
    switch_frame = 65
elif recording == "stack_recordings/episode_156":
    switch_frame = 80
else:
    switch_frame = 1e6
    
for i in range(len(image_recording)):
    if i < switch_frame:
        color_index = 2
    else:
        color_index = 0
    mask = get_mask(image_recording[i], threshold, color_index)
    masks[i] = mask

print(np.mean(masks) * 100, "percent of pixels fg")
if folder_format == 'max':
    mask_fn = flow_recording_fn.replace("_img.npz","_mask.npz")
else:
    mask_fn = flow_recording_fn.replace(".npz","_mask.npz")
np.savez(mask_fn, mask=masks)
print("Saved to",mask_fn)

In [ ]:
fig, ax = plt.subplots(1,1)
line = ax.imshow(masks[25])
ax.set_axis_off()

def update(i):
    image = image_recording[i].copy()
    mask = masks[i]
    image[np.logical_not(mask)] = 255,255,255
    line.set_data(image)
    fig.canvas.draw_idle()
    
slider_i2 = widgets.IntSlider(min=0,max=num_frames,step=1,value=25,
                             layout=Layout(width='70%'))

interact(update, i=slider_i2)

In [ ]:
# This should select a bounding box.

index_A = 25
index_B = 5

fig, ((ax, ax2), (ax3, ax4)) = plt.subplots(2, 2)
ax.imshow(image_recording[index_A])
ax.set_axis_off()
tmp = image_recording[index_A].copy()
tmp[20:60,35:55] = 255

ax2.imshow(tmp)
ax2.set_axis_off()

ax3.imshow(image_recording[index_B])

